In [ ]:
import os
import numpy as np
import matplotlib.image as mpimg
import cv2
from docopt import docopt
from IPython.display import HTML
from IPython.core.display import Video
from moviepy.editor import VideoFileClip
from CameraCalibration import CameraCalibration
from Thresholding import *
from PerspectiveTransformation import *
from LaneLines import *

class FindLaneLines:
    def __init__(self):
        """ Init Application"""
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, img):
        out_img = np.copy(img)
        img = self.calibration.undistort(img)
        img = self.transform.forward(img)
        img = self.thresholding.forward(img)
        img = self.lanelines.forward(img)
        img = self.transform.backward(img)

        out_img = cv2.addWeighted(out_img, 1, img, 0.6, 0)
        out_img = self.lanelines.plot(out_img)
        return out_img

    def process_image(self, input_path, output_path):
        img = mpimg.imread(input_path)
        out_img = self.forward(img)
        mpimg.imsave(output_path, out_img)

    def process_video(self, input_path, output_path):
        clip = VideoFileClip(input_path)
        out_clip = clip.fl_image(self.forward)
        out_clip.write_videofile(output_path, audio=False)

def laneLineDetection(videoPath):
    findLaneLines = FindLaneLines()
    findLaneLines.process_video(videoPath,'./outputs/'+os.path.splitext(videoPath)[0].split('/')[-1]+'out.mp4')
    videoPath = './outputs/'+os.path.splitext(videoPath)[0].split('/')[-1]+'out.mp4'

In [ ]:
from flask import Flask, render_template, request, url_for, redirect

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        file = request.files['file']
        file.save('./uploads/' + file.filename)
        laneLineDetection('./uploads/' + file.filename)
        return 'File saved!'
        #return redirect(url_for("play_video",path='./outputs/' + file.filename.replace('.mp4','')+'out.mp4'))
    return '''<!DOCTYPE html>
<html>
<head>
    <title>Lane Line Detection</title>
</head>
<body style = "font-family: Arial, sans-serif;">
    <form action="/" method="post" enctype="multipart/form-data" style = "display: flex;
    flex-direction: column;
    align-items: center;">
        <input type="file" name="file" style = "margin-bottom: 10px;">
        <input type="submit" value="Upload" style = "background-color: #4CAF50;
    color: white;
    padding: 12px 20px;
    border: none;
    border-radius: 4px;">
    </form>
</body>
</html>
'''

@app.route('/play-video')
def play_video():
    path = request.args.get('path').replace('%2','/')
    # Get the URL for the video file
    video_url = url_for(path)
    # Render the HTML player template
    print(video_url)
    return render_template('''<!DOCTYPE html>
<html>
  <body>
   <video width="640" height="360" controls>
		<source src="{{'./outputs/'}}" type="video/mp4">
   </video>
</body>
</html>
''', video_url=video_url)

if __name__ == '__main__':
    app.run()
